# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

# Data Preprocessing

In [2]:
# Load data, selecting data from the 25840th till the 51657th row
df = pd.read_csv(
    '100_Portfolios_10x10_Daily.csv', 
    skiprows=25817, 
    nrows=(51613 - 25818), 
    header=0,    # Set first row as the column
    index_col=0  # Set date as the index
)

# Changing the index to datetime format
df.index = pd.to_datetime(df.index, format='%Y%m%d')

# Retrieving data dated between 2022-12-29 and 2024-07-31
df = df[(df.index >= '2022-12-29') & (df.index <= '2024-07-31')]

# Drop 'ME9 BM10' due to presence of missing values
df = df.drop('ME9 BM10', axis=1)

df.head()

,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,ME1 BM5,ME1 BM6,ME1 BM7,ME1 BM8,ME1 BM9,SMALL HiBM,...,BIG LoBM,ME10 BM2,ME10 BM3,ME10 BM4,ME10 BM5,ME10 BM6,ME10 BM7,ME10 BM8,ME10 BM9,BIG HiBM
2022-12-29,3.61,3.91,5.68,4.66,3.95,4.39,3.51,4.88,2.82,2.80,...,2.19,1.94,1.25,1.14,1.12,1.30,0.76,1.24,1.38,1.04
2022-12-30,3.57,3.06,4.97,1.58,2.36,1.87,2.29,2.49,1.69,1.35,...,-0.42,-0.45,-0.36,-0.34,-0.50,-0.08,0.20,0.13,-0.23,-0.20
2023-01-03,-1.04,0.10,0.86,2.24,3.95,1.57,6.37,2.27,1.88,1.91,...,-0.47,0.57,0.30,-0.86,0.08,0.54,-0.73,-1.54,0.62,0.15
2023-01-04,3.58,3.14,5.35,4.73,2.72,4.14,3.74,2.38,2.18,2.89,...,1.28,1.04,0.81,0.29,0.58,2.28,0.68,0.32,1.56,1.66
2023-01-05,0.36,-1.45,0.48,0.54,1.93,-0.08,-0.35,0.97,1.39,0.39,...,-1.68,-1.11,-1.07,-1.81,-0.60,-0.62,-0.07,0.41,-0.72,-0.40


In [3]:
# Confirm data structure & absence of missing values
df.replace([-99.99, -999], np.nan, inplace=True)  # inplace = True to modify the original DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 398 entries, 2022-12-29 to 2024-07-31
Data columns (total 99 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SMALL LoBM  398 non-null    float64
 1   ME1 BM2     398 non-null    float64
 2   ME1 BM3     398 non-null    float64
 3   ME1 BM4     398 non-null    float64
 4   ME1 BM5     398 non-null    float64
 5   ME1 BM6     398 non-null    float64
 6   ME1 BM7     398 non-null    float64
 7   ME1 BM8     398 non-null    float64
 8   ME1 BM9     398 non-null    float64
 9   SMALL HiBM  398 non-null    float64
 10  ME2 BM1     398 non-null    float64
 11  ME2 BM2     398 non-null    float64
 12  ME2 BM3     398 non-null    float64
 13  ME2 BM4     398 non-null    float64
 14  ME2 BM5     398 non-null    float64
 15  ME2 BM6     398 non-null    float64
 16  ME2 BM7     398 non-null    float64
 17  ME2 BM8     398 non-null    float64
 18  ME2 BM9     398 non-null    float64
 19  ME2 BM10  

# Defining Parameter

In [4]:
# Size of Scaling Window
training_window_size = 252

# Range of Lambda Values for Cross-Validation
alpha = np.logspace(-8, 8, 100)
best_alpha_LASSO = 0
best_alpha_Ridge = 0

# Test dates
return_dates = df.index.to_list()[252:]

# To store results for predicted returns
predicted_returns = {
    'EW': [],
    'MinVar': [],
    'Ridge': [],
    'LASSO': [],
    'Constrained-LASSO': []
}

# To store results for sharpe ratio
sharpe_ratios = {
    'EW': 0,
    'MinVar': 0,
    'Ridge': 0,
    'LASSO': 0,
    'Constrained-LASSO': 0
}

# Defining Function

In [5]:
# Sharpe Ratio
def sharpe_ratio(test_return):
    avg_test_return = np.mean(test_return)
    stddev_test_return = np.std(test_return)
    return avg_test_return / stddev_test_return

In [6]:
# Equation for adjusted weightage for different models (OLS, LassoCV, RidgeCV)
def w_adjusted(train_data, model):
    # Declare global variables to store best alpha for LassoCV and RidgeCV
    global best_alpha_Ridge, best_alpha_LASSO

    # Demean training data without standardization
    train_data_demeaned = scale(train_data, with_mean=True, with_std=False)

    # Number of stocks
    N_stocks = len(train_data.columns)

    # Define equal weights
    equal_weight = np.ones(N_stocks) / N_stocks

    # Define matrix, N, for weight adjustments (sum constraint)
    N = np.eye(N_stocks - 1)  # Identity matrix of N_stocks - 1 size
    N = np.vstack([N, -1 * np.ones(N_stocks - 1)])  # Append row for sum constraint, making N matrix (N_stocks, N_stocks - 1)

    y = train_data_demeaned @ equal_weight
    X = train_data_demeaned @ N

    if model == 'LinearRegression':
        current_model = LinearRegression(fit_intercept=False).fit(X, y)
    if model == 'RidgeCV':
        current_model = RidgeCV(alphas=alpha, cv=5, fit_intercept=False).fit(X, y)
        best_alpha_Ridge = current_model.alpha_
    if model == 'LassoCV':
        current_model = LassoCV(alphas=alpha, cv=5, max_iter=5000, fit_intercept=False).fit(X, y)
        best_alpha_LASSO = current_model.alpha_
        
    beta = current_model.coef_

    w_adjusted = equal_weight - np.dot(N, beta)

    return w_adjusted



# OLS (First 6 assets): Adjusted Weightage & Predicted Return on 2024-01-02

In [7]:
# To retrieve data for the first 252 rows, and the first 6 columns (i.e., assets)
train_data = df.iloc[0:252, :6]

# To calculate the adjusted weightage, based on the model used
adj_weightage = w_adjusted(train_data, 'LinearRegression')

# Retrieve the real-time return for the next day
next_day_return = df.iloc[252, :6].values

# Calculate next-day (i.e., 2024-01-02) return using the predicted weightage
portfolio_return = np.dot(adj_weightage, next_day_return)

print('Adjusted weightage for first 6 assets: ', adj_weightage)
print('Min-Var portfolio return [first 6 assets]: ', portfolio_return)

Adjusted weightage for first 6 assets:  [ 0.13610383  0.25476295 -0.0921509   0.15418786  0.22985689  0.31723938]
Min-Var portfolio return [first 6 assets]:  0.7731660277814977


# Equal-Weightage Portfolio (99 Assets)

In [8]:
for i in range (training_window_size, len(df)):
    # Amount of assets
    N_stocks = len(df.columns)

    # Equal weights for the portfolio
    equal_weight = np.ones(N_stocks) / N_stocks

    # Retrieve the real-time return for the next day
    next_day_return = df.iloc[i].values

    # Calculate next-day (i.e., 2024-01-02) return using the predicted weightage
    portfolio_return = np.dot(equal_weight, next_day_return)

    # Store predicted returns
    predicted_returns['EW'].append(portfolio_return)

print('EW Return Table:\n', pd.DataFrame(data=predicted_returns, index=return_dates, columns=['EW']))

sharpe_ratios['EW'] = sharpe_ratio(predicted_returns['EW'])
print('EW Sharpe Ratio: ', sharpe_ratios['EW'])

EW Return Table:
                   EW
2024-01-02 -0.376465
2024-01-03 -2.217576
2024-01-04 -0.064040
2024-01-05 -0.021919
2024-01-08  1.697778
...              ...
2024-07-25  1.122424
2024-07-26  1.614141
2024-07-29 -0.727677
2024-07-30  0.229192
2024-07-31  0.571313

[146 rows x 1 columns]
EW Sharpe Ratio:  0.06540829983611159


# Min-Var Portfolio (99 Assets)

In [9]:
for i in range(training_window_size, len(df)):

    # Defining training period
    train_data = df.iloc[i-training_window_size:i]

    # To calculate the adjusted weightage, based on the model used
    adj_weightage = w_adjusted(train_data, 'LinearRegression')

    # Retrieve the real-time return for the next day
    next_day_return = df.iloc[i].values

    # Calculate next-day (i.e., 2024-01-02) return using the predicted weightage
    portfolio_return = np.dot(adj_weightage, next_day_return)

    # Store predicted returns
    predicted_returns['MinVar'].append(portfolio_return)

print('MinVar Return Table:\n', pd.DataFrame(data=predicted_returns, index=return_dates, columns=['MinVar']))

sharpe_ratios['MinVar'] = sharpe_ratio(predicted_returns['MinVar'])
print('MinVar Sharpe Ratio: ', sharpe_ratios['MinVar'])

MinVar Return Table:
               MinVar
2024-01-02  1.742367
2024-01-03 -0.812425
2024-01-04 -0.301904
2024-01-05  0.114314
2024-01-08  0.961189
...              ...
2024-07-25 -0.616636
2024-07-26  0.943585
2024-07-29 -0.103261
2024-07-30 -0.457016
2024-07-31  1.260733

[146 rows x 1 columns]
MinVar Sharpe Ratio:  0.00758241850160112


# Ridge Portfolio (99 Assets)

In [10]:
for i in range(training_window_size, len(df)):

    # Defining training period
    train_data = df.iloc[i-training_window_size:i]

    # To calculate the adjusted weightage, based on the model used
    adj_weightage = w_adjusted(train_data, 'RidgeCV')

    # Retrieve the real-time return for the next day
    next_day_return = df.iloc[i].values

    # Calculate next-day (i.e., 2024-01-02) return using the predicted weightage
    portfolio_return = np.dot(adj_weightage, next_day_return)

    # Store predicted returns
    predicted_returns['Ridge'].append(portfolio_return)

print("Final lambda (alpha) used in RidgeCV: ", best_alpha_Ridge, '\n')
print('Ridge Return Table:\n', pd.DataFrame(data=predicted_returns, index=return_dates, columns=['Ridge']))

sharpe_ratios['Ridge'] = sharpe_ratio(predicted_returns['Ridge'])
print('Ridge Sharpe Ratio: ', sharpe_ratios['Ridge'])

Final lambda (alpha) used in RidgeCV:  104.76157527896662 

Ridge Return Table:
                Ridge
2024-01-02  1.522888
2024-01-03 -0.491125
2024-01-04 -0.399170
2024-01-05  0.132758
2024-01-08  0.834212
...              ...
2024-07-25 -0.359387
2024-07-26  1.388059
2024-07-29  0.126964
2024-07-30 -0.184015
2024-07-31  0.633287

[146 rows x 1 columns]
Ridge Sharpe Ratio:  0.1013248696436384


# LASSO Portfolio (99 Assets)

In [11]:
for i in range(training_window_size, len(df)):

    # Defining training period
    train_data = df.iloc[i-training_window_size:i]

    # To calculate the adjusted weightage, based on the model used
    adj_weightage = w_adjusted(train_data, 'LassoCV')

    # Retrieve the real-time return for the next day
    next_day_return = df.iloc[i].values

    # Calculate next-day (i.e., 2024-01-02) return using the predicted weightage
    portfolio_return = np.dot(adj_weightage, next_day_return)

    # Store predicted returns
    predicted_returns['LASSO'].append(portfolio_return)
    
print("Final lambda (alpha) used in LassoCV: ", best_alpha_LASSO, '\n')
print('LASSO Return Table:\n', pd.DataFrame(data=predicted_returns, index=return_dates, columns=['LASSO']))

sharpe_ratios['LASSO'] = sharpe_ratio(predicted_returns['LASSO'])
print('LASSO Sharpe Ratio: ', sharpe_ratios['LASSO'])

Final lambda (alpha) used in LassoCV:  0.029150530628251816 

LASSO Return Table:
                LASSO
2024-01-02  1.405719
2024-01-03 -0.384078
2024-01-04 -0.604772
2024-01-05  0.114279
2024-01-08  0.767850
...              ...
2024-07-25 -0.071232
2024-07-26  1.641609
2024-07-29  0.125146
2024-07-30 -0.225694
2024-07-31  0.934691

[146 rows x 1 columns]
LASSO Sharpe Ratio:  0.11058695714342875


# Exploration: Constrained Mean-Variance Portfolio

### Defining Parameters

In [12]:
# Set limits on the total proportion of the portfolio allocated to high-volatility and low-volatility assets respectively
high_vol_limit = 0.4
low_vol_limit = 0.6

# Define the maximum weight that can be assigned to each high-volatility or low-volatility asset
upper_bound_high_vol = 0.1
upper_bound_low_vol = 1.0

# Tradeoff between risk and return (i.e., risk appetite)
risk_aversion_param = 0.5

In [13]:
# Extract Data
# Retrive daily return of each asset
asset_returns = df.values

# Using equal-weightage returns as the benchmark for market_returns
market_returns = np.mean(asset_returns, axis=1)

# Compute Mean Returns and Covariance Matrix
mean_returns = np.mean(asset_returns, axis=0)    # Expected return for each asset over the entire time period
cov_matrix = np.cov(asset_returns, rowvar=False) # Covariance matrix of asset returns (to calculate portfolio risk)

In [14]:
# Define Asset Classes
betas = [np.cov(asset_returns[:, i], market_returns)[0][1] / np.var(market_returns) for i in range(asset_returns.shape[1])]
high_vol_assets = [i for i, beta in enumerate(betas) if beta > 1]
low_vol_assets = [i for i, beta in enumerate(betas) if beta <= 1]

### Create Model

In [15]:
# Create Model
def optimize_portfolio(mean_returns, cov_matrix, high_vol_assets, low_vol_assets, regularization_type="None"):
    num_assets = len(mean_returns)
    model = gp.Model("Constrained Portfolio Optimization")

    # Variables: weights for each asset
    weights = model.addVars(num_assets, lb=0, name="weights")

    # Objective: Minimize risk_aversion_param * (w.T * cov_matrix * w) - (1 - risk_aversion_param) * (w.T * mean_returns)
    portfolio_return = gp.quicksum(weights[i] * mean_returns[i] for i in range(num_assets))
    portfolio_risk = gp.quicksum(weights[i] * cov_matrix[i, j] * weights[j] for i in range(num_assets) for j in range(num_assets))

    # Add LASSO or Ridge Regularization
    # Penalty level for Constrained-LASSO and Constrained-Ridge, retrieved from previous results of LassoCV (best_alpha_LASSO) and RidgeCV (best_alpha_Ridge)
    if regularization_type == "LASSO":
        # Auxiliary variables for LASSO regularization
        abs_weights = model.addVars(num_assets, name="abs_weights")
        # Add constraints to link abs_weights with weights
        for i in range(num_assets):
            model.addConstr(abs_weights[i] >= weights[i], name=f"abs_weight_pos_{i}")
            model.addConstr(abs_weights[i] >= -weights[i], name=f"abs_weight_neg_{i}")
        regularization_term = gp.quicksum(best_alpha_LASSO * abs_weights[i] for i in range(num_assets))
    elif regularization_type == "Ridge":
        regularization_term = gp.quicksum(best_alpha_Ridge * (weights[i] * weights[i]) for i in range(num_assets))
    else:
        regularization_term = 0

    # Set Objective: Minimize risk + regularization - expected return
    model.setObjective(risk_aversion_param * portfolio_risk - (1 - risk_aversion_param) * portfolio_return + regularization_term, GRB.MINIMIZE)

    # Constraint: Fully invested constraint (sum(weights) = 1)
    model.addConstr(weights.sum() == 1, "fully_invested")

    # Constraints: Class constraints
    high_vol_weight = gp.quicksum(weights[i] for i in high_vol_assets)
    low_vol_weight = gp.quicksum(weights[i] for i in low_vol_assets)
    model.addConstr(high_vol_weight <= high_vol_limit, "high_vol_limit")
    model.addConstr(low_vol_weight >= low_vol_limit, "low_vol_limit")

    # Bounds for each asset class
    for i in range(num_assets):
        if i in high_vol_assets:
            weights[i].setAttr(GRB.Attr.UB, upper_bound_high_vol)
        else:
            weights[i].setAttr(GRB.Attr.UB, upper_bound_low_vol)

    # Solve
    model.optimize()

    if model.status == GRB.OPTIMAL:
        optimal_weights = model.getAttr('x', weights)
        return [optimal_weights[i] for i in range(num_assets)]
    else:
        print("No optimal solution found.")
        return None

### LASSO Regularization with Portfolio Weight Constraints

In [16]:
for i in range(training_window_size, len(df)):
    # Define training period
    train_data = df.iloc[i-training_window_size:i]

    # Compute mean returns and covariance matrix for training period
    mean_returns_train = np.mean(train_data, axis=0)
    cov_matrix_train = np.cov(train_data, rowvar=False)

    # Optimize portfolio for training period with LASSO regularization
    adjusted_weights = optimize_portfolio(mean_returns_train, cov_matrix_train, high_vol_assets, low_vol_assets, regularization_type="LASSO")

    # Calculate next-day return using predicted weights
    next_day_return = asset_returns[i]
    if adjusted_weights is not None:
        portfolio_return = np.dot(adjusted_weights, next_day_return)
    else:
        predicted_return = 0  # Fallback if no optimal solution found

    # Store predicted returns
    predicted_returns['Constrained-LASSO'].append(portfolio_return)

# Display Predicted Returns and Sharpe Ratio
print('Constrained-LASSO Return Table:\n', pd.DataFrame(data=predicted_returns, index=return_dates, columns=['Constrained-LASSO']))

sharpe_ratios['Constrained-LASSO'] = sharpe_ratio(predicted_returns['Constrained-LASSO'])
print('Constrained-LASSO Sharpe Ratio: ', sharpe_ratios['Constrained-LASSO'])

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-09
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 201 rows, 198 columns and 594 nonzeros
Model fingerprint: 0xebb72b4e
Model has 4950 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-03, 1e-01]
  QObjective range [6e-01, 7e+00]
  Bounds range     [1e-01, 1e+00]
  RHS range        [4e-01, 1e+00]
Presolve removed 198 rows and 99 columns
Presolve time: 0.01s
Presolved: 3 rows, 99 columns, 198 nonzeros
Presolved model has 4950 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 98
 AA' NZ     : 5.049e+03
 Factor NZ  : 5.151e+03
 Factor Ops : 3.486e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter    

/var/folders/j5/m90b2sqx6k9dr4l9ys0fgx300000gn/T/ipykernel_25296/2123043764.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  portfolio_return = gp.quicksum(weights[i] * mean_returns[i] for i in range(num_assets))


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 201 rows, 198 columns and 594 nonzeros
Model fingerprint: 0x89bc37c9
Model has 4950 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-18, 9e-02]
  QObjective range [6e-01, 7e+00]
  Bounds range     [1e-01, 1e+00]
  RHS range        [4e-01, 1e+00]
Presolve removed 198 rows and 99 columns
Presolve time: 0.00s
Presolved: 3 rows, 99 columns, 198 nonzeros
Presolved model has 4950 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 98
 AA' NZ     : 5.049e+03
 Factor NZ  : 5.151e+03
 Factor Ops : 3.486e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.29539456e+04 -6.

# Result

In [17]:
# Predicted Returns for Each Portfolio
predicted_returns_df = pd.DataFrame(data=predicted_returns, index=return_dates)
print('Summary Return:\n', predicted_returns_df)


Summary Return:
                   EW    MinVar     Ridge     LASSO  Constrained-LASSO
2024-01-02 -0.376465  1.742367  1.522888  1.405719           0.529651
2024-01-03 -2.217576 -0.812425 -0.491125 -0.384078          -0.651303
2024-01-04 -0.064040 -0.301904 -0.399170 -0.604772          -0.433286
2024-01-05 -0.021919  0.114314  0.132758  0.114279           0.204586
2024-01-08  1.697778  0.961189  0.834212  0.767850           0.883565
...              ...       ...       ...       ...                ...
2024-07-25  1.122424 -0.616636 -0.359387 -0.071232           0.015803
2024-07-26  1.614141  0.943585  1.388059  1.641609           1.251001
2024-07-29 -0.727677 -0.103261  0.126964  0.125146          -0.010590
2024-07-30  0.229192 -0.457016 -0.184015 -0.225694           0.374379
2024-07-31  0.571313  1.260733  0.633287  0.934691           0.610266

[146 rows x 5 columns]


In [18]:
# Sharpe Ratio
sharpe_ratios_df = pd.DataFrame(data=sharpe_ratios, index=['Sharpe Ratio'])
print(sharpe_ratios_df)

                    EW    MinVar     Ridge     LASSO  Constrained-LASSO
Sharpe Ratio  0.065408  0.007582  0.101325  0.110587           0.171222
